In [ ]:
# Install necessary packages
!pip install yahoo_fin
!pip install yfinance

In [1]:
# import libraries
import yfinance as yf
import pandas as pd

In [10]:
# Company ticker
TICKERS = ['TSLA']

In [3]:
def get_financial_statements(ticker, save_to_csv = False):
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet
    income_statement = stock.income_stmt
    cashflow_statement = stock.cashflow
    years = balance_sheet.columns
    
    if save_to_csv:
    # Save to CSV
        balance_sheet.to_csv(f"{ticker}_balance_sheet.csv")
        income_statement.to_csv(f"{ticker}_income_statement.csv")
        cashflow_statement.to_csv(f"{ticker}_cashflow_statement.csv")
    
    return balance_sheet,income_statement,cashflow_statement, years

## Profitability Criteria

In [4]:
def probability_score(balance_sheet,income_statement,cashflow_statement, years):
    # Score 1 - Positive RoA or we check NI. Beacuse Assets are always +ve
    net_income = income_statement[years[0]]['Net Income']
    
    # Score 2 - Postive RoA in cy - more than this higher than py NI would worth to see.
    net_income_py = income_statement[years[1]]['Net Income']

    
    # Score 3 -  Positive operating cashflow in the cy
    op_cf = cashflow_statement[years[0]]['Cash Flow From Continuing Operating Activities']

    
    # Score 4 - Change in RoA (Revene/Total Assests)-> when aking BS stuufs it is always better to get Avg.
    avg_assets = (balance_sheet[years[0]]['Total Assets']+ 
                  balance_sheet[years[1]]['Total Assets'])/2
    avg_assets_py = (balance_sheet[years[1]]['Total Assets']+ 
                     balance_sheet[years[1]]['Total Assets'])/2
    
    RoA = net_income/avg_assets
    RoA_py = net_income_py/ avg_assets_py

    
    # Score 5 - Accruals (Quality of the Earnings)  (operating cashflow/Total Assests)-RoA (Here we are comparring cash based performance to accruals based perfomance)
    total_assets = balance_sheet[years[0]]['Total Assets']
    accruals_ratio = (op_cf/total_assets) - RoA

    return sum([
        net_income > 0,
        net_income > net_income_py,
        op_cf > 0,
        RoA > RoA_py,
        accruals_ratio > 0
    ])

## Leverage, Liquidity, and Source of Funds Criteria

In [5]:
def leverage_score(balance_sheet,years):
    # Score 6 - Rather than checking whether long-term debt has reduced, i would prefer to give 1 point if debt ratio is below 0.4
    try:
        LT_debt = balance_sheet[years[0]]['Long Term Debt']
        total_assets = balance_sheet[years[0]]['Total Assets']
        debt_ratio = LT_debt / total_assets
        debt_ratio_score = 1 if debt_ratio < 0.4 else 0
    except:
        debt_ratio_score = 1
    # or I can see the changes in debt_ratio.....
    
    # Score 7 - Current Ratio (Or might sometimes check the changes in current Ratio)
    current_assets = balance_sheet[years[0]]['Current Assets']
    current_liabilities = balance_sheet[years[0]]['Current Liabilities']
    current_ratio = current_assets / current_liabilities
    current_ratio_score = 1 if current_ratio > 1 else 0
    
    # Score 8 New shares issued?
    
    return debt_ratio_score + current_ratio_score

## Operating Efficiency Criteria

In [6]:
def operating_efficiency(balance_sheet,income_statement, years):
    # Score 8 - change in gross margin
    gp = income_statement[years[0]]['Gross Profit']
    gp_py = income_statement[years[1]]['Gross Profit']
    
    revenue = income_statement[years[0]]['Total Revenue']
    revenue_py = income_statement[years[1]]['Total Revenue']
    
    gp_margin = (gp/revenue)
    gp_margin_py = gp_py/revenue_py
    
    # Score 9 Change in Assets Turnover
    avg_assets = (balance_sheet[years[0]]['Total Assets']+
                  balance_sheet[years[1]]['Total Assets'])/2
    
    avg_assets_py = (balance_sheet[years[1]]['Total Assets']+
                  balance_sheet[years[2]]['Total Assets'])/2
    assets_turnover = revenue/avg_assets
    assets_turnover_py = revenue_py/avg_assets_py
    
    return sum([
        gp_margin > gp_margin_py,
        assets_turnover > assets_turnover_py
    ])

In [7]:
# Calculate P/E artio with error handling.
def get_pe_ratio(stock):
    try:
        pe_ratio = stock.info.get('trailingPE')
        return pe_ratio if pe_ratio is not None else "N/A"
    except Exception as e:
        return f"Error: {str(e)}"

In [8]:
def analyse_stock(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet, income_statement, cashflow_statement, years = get_financial_statements(ticker)
        
        pe_ratio = get_pe_ratio(stock)
        profitability_score = probability_score(balance_sheet,income_statement,cashflow_statement, years)
        leveragescore = leverage_score(balance_sheet, years)
        operating_efficiency_score = operating_efficiency(balance_sheet,income_statement, years)
        
        return {
            'Ticker': ticker,
            'PE Ratio': pe_ratio,
            'Profitability': profitability_score,
            'Leverage': leveragescore,
            'Operating Efficiency': operating_efficiency_score
        }
    except Exception as e:
        print(f"{ticker}: Something went wrong - {str(e)}")
        return None

def analyse_stocks(tickers):
    summary = pd.DataFrame(columns=['Ticker', 'PE Ratio', 'Profitability', 'Leverage', 'Operating Efficiency'])
    
    for ticker in tickers:
        result = analyse_stock(ticker)
        if result:
            summary = pd.concat([summary, pd.DataFrame([result])], ignore_index=True)
            print(f"{ticker} added.")
        else:
            print(f"{ticker} skipped due to errors.")
    
    if not summary.empty:
        summary['Total_Score'] = summary[['Profitability', 'Leverage', 'Operating Efficiency']].sum(axis=1)
    return summary


In [11]:
result_summary = analyse_stocks(TICKERS)
result_summary

TSLA added.


,Ticker,PE Ratio,Profitability,Leverage,Operating Efficiency,Total_Score
0,TSLA,59.72269,4,2,0,6
